### 문제 7-1 : LangGraph ReAct Agent 실습 연습문제 (Vector DB + Tool 연동) 


In [1]:
from dotenv import load_dotenv
from textwrap import dedent
import warnings
from langchain_community.vectorstores import FAISS
from langchain_ollama  import OllamaEmbeddings
from langchain_core.tools import tool
from typing import List
from langchain_community.tools import TavilySearchResults
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.prebuilt import tools_condition

warnings.filterwarnings("ignore")

load_dotenv()

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# Tool 정의 
@tool
def search_menu(query: str) -> List[str]:
    """카페 메뉴에서 정보를 검색합니다."""
    docs = menu_db.similarity_search(query, k=6)

    formatted_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

    if len(docs) > 0:
        return formatted_docs
    
    return "관련 메뉴 정보를 찾을 수 없습니다."



# Tool 정의 
@tool
def search_web(query: str) -> List[str]:
    """데이터베이스에 존재하지 않는 정보 또는 최신 정보를 인터넷에서 검색합니다."""

    tavily_search = TavilySearchResults(max_results=3)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in docs
        ]
    )

    if len(docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."


# LLM 모델 
llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)

# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.2
# )

# 도구 목록
tools = [search_menu, search_web]
llm_with_tools = llm.bind_tools(tools=tools)

system_prompt = dedent("""
당신은 사용자 질문에 답변하는 카페 AI 어시스턴트입니다.  
제공된 도구들을 활용해 정확한 정보를 전달해야 합니다.

[작동 원칙]
1. 질문 이해: 사용자의 질문을 정확히 파악하세요
2. 도구 활용: 필요한 정보는 반드시 제공된 도구로 조회
3. 출처 명시: 도구 사용 후 즉시 아래 형식으로 출처 표기
4. 최종 답변은 질문과 직접 관련된 명확한 내용으로 구성하고, 불필요한 정보는 포함하지 마세요.


[도구 사용 형식]
액션: 도구_이름  
액션 입력: 도구에_넘길_입력값  

[출처 표기 형식]
[출처: 도구_이름 | 문서_제목/항목명 | URL/파일경로]

[예시 1 - 메뉴 검색]
액션: search_menu  
액션 입력: 아메리카노  

(도구 실행 후)  
[출처: search_menu | 아메리카노 | ../data/cafe_menu_data.txt]  
아메리카노 정보: 아메리카노 4,500원...

[예시 2 - 웹 검색]  
액션: search_web  
액션 입력: AI 역사  

(도구 실행 후)  
[출처: search_web | AI 역사 | https://ko.wikipedia.org/wiki/인공지능]  
AI 역사는 1950년대부터...

[주의사항]
1. 도구가 필요없는 질문은 직접 답변
2. 모든 사실 정보는 반드시 출처 동반
3. 출처 없이는 어떠한 정보도 제공하지 말 것
4. 최종 답변은 질문과 직접 관련된 명확한 내용으로 구성하고, 불필요한 정보는 포함하지 마세요.

""")

class GraphState(MessagesState):
    pass

# 노드 구성 
def call_model(state: GraphState):
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}
tool_node = ToolNode(tools=tools)

# 그래프 구성
builder = StateGraph(GraphState)
builder.add_node("agent", call_model)
builder.add_node("tools", tool_node)

builder.add_edge(START, "agent")
builder.add_conditional_edges(
    "agent",
    tools_condition,
)
builder.add_edge("tools", "agent")

graph = builder.compile()

In [2]:
# 테스트 시나리오
print("--- 아메리카노 비교 ---")
inputs_1 = {"messages": [SystemMessage(content=system_prompt), HumanMessage(content="아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.")]}
messages_1 = graph.invoke(inputs_1)
for m in messages_1['messages']:
    if isinstance(m, AIMessage) and not m.tool_calls and m.content:
        print(m.content)

print("\n--- 라떼 종류 문의 ---")
inputs_2 = {"messages": [SystemMessage(content=system_prompt), HumanMessage(content="라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?")]}
messages_2 = graph.invoke(inputs_2)
for m in messages_2['messages']:
    if isinstance(m, AIMessage) and not m.tool_calls and m.content:
        print(m.content)

print("\n--- 디저트 문의 ---")
inputs_3 = {"messages": [SystemMessage(content=system_prompt), HumanMessage(content="디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요.")]}
messages_3 = graph.invoke(inputs_3)
for m in messages_3['messages']:
    if isinstance(m, AIMessage) and not m.tool_calls and m.content:
        print(m.content)

# 도구가 필요 없는 질문 예시
print("\n--- 일반적인 질문 ---")
inputs_4 = {"messages": [SystemMessage(content=system_prompt), HumanMessage(content="langgraph란?")]}
messages_4 = graph.invoke(inputs_4)
for m in messages_4['messages']:
    if isinstance(m, AIMessage) and not m.tool_calls and m.content:
        print(m.content)


--- 아메리카노 비교 ---
아메리카노와 아이스 아메리카노는 기본적인 재료는 동일하지만, 제공되는 온도에 따라 큰 차이가 있습니다.

1. **아메리카노**:
   - 가격: 4,500원
   - 종류: 에스프레소와 뜨거운 물로 만들어집니다.

2. **아이스 아메리카노**:
   - 가격: 4,500원
   - 종류: 에스프레소와 차가운 물, 그리고 얼음으로 만들어집니다.

결론적으로 두 음료의 가격은 동일하지만, 아메리카노는 뜨거운, 아이스 아메리카노는 차가운 형태로 판매됩니다. 

[출처: search_menu | 아메리카노 | ../data/cafe_menu_data.txt]  
[출처: search_menu | 아이스 아메리카노 | ../data/cafe_menu_data.txt]

--- 라떼 종류 문의 ---
라떼 종류에는 다음과 같은 메뉴가 있습니다:

1. **카페라떼**
   - 가격: ₩5,500
   - 주요 원료: 에스프레소, 스팀 밀크
   - 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다.

2. **바닐라 라떼**
   - 가격: ₩6,000
   - 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽
   - 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 풍성한 맛을 즐길 수 있습니다.

3. **녹차 라떼**
   - 가격: ₩5,800
   - 주요 원료: 말차 파우더, 스팀 밀크, 설탕
   - 설명: 고급 말차 파우더와 부드러운 스팀 밀크로 만든 건강한 음료입니다. 녹차의 은은한 쓴맛과 우유의 부드러움이 조화를 이루며, 항산화 성분이 풍부합니다.

4. **카라멜 마키아토**
   - 가격: ₩6,500
   - 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림
   - 설명: 스팀 밀크 위에 에스프레소를 

C:\Users\user\AppData\Local\Temp\ipykernel_9292\389995913.py:53: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=3)


LangGraph는 LangChain에서 개발한 오픈 소스 AI 에이전트 프레임워크입니다. 이 프레임워크는 복잡한 생성적 AI 에이전트 워크플로우를 구축, 배포 및 관리하기 위한 도구와 라이브러리를 제공합니다. LangGraph는 그래프 기반 아키텍처를 사용하여 인공지능 워크플로우를 보다 효율적으로 모델링하고 관리할 수 있게 해줍니다.

주요 특징은 다음과 같습니다:
- 상태를 유지하는 오케스트레이션 프레임워크로, 에이전트 기반 애플리케이션에 사용됩니다.
- 대규모 언어 모델(LLMs)을 효과적으로 운영할 수 있는 인프라를 제공합니다.
- 복잡한 관계를 모델링하여 AI 에이전트의 의사결정을 향상시킵니다.

이 프레임워크는 대화형 에이전트, 복잡한 작업 자동화 및 사용자 경험을 지원하는 LLM 기반 솔루션을 구축할 수 있는 기반을 제공합니다.

[출처: search_web | langgraph | https://www.ibm.com/think/topics/langgraph]
